In [908]:
class Node:
    def __init__(self, name, data, power):
        self.name = name
        self.data = data
        self.power = power

class Gateway:
    def __init__(self, name, power_setup, storage_capacity):
        self.name = name
        self.power_setup = power_setup
        self.storage_capacity = storage_capacity

class Link:
    def __init__(self, node, gateway, power):
        self.node = node
        self.gateway = gateway
        self.power = power


In [909]:
DEBUG = False

def exact_cover(nodes, gateways, links):
    # Step 1: Initialize Sets and Data Structures
    node_set = set(nodes)
    gateway_set = set(gateways)
    node_gateway_links = {node: set() for node in node_set}
    gateway_capacity = {gateway: gateway.storage_capacity for gateway in gateway_set}
    node_power_budget = {node: node.power for node in node_set}
    selected_gateways = set()
    selected_nodes = set()
    selected_links = set()
    min_power_consumption = float('inf')
    optimal_gateways = None
    optimal_nodes = None
    optimal_links = None
    last_optimal_gateways = None
    last_optimal_nodes = None
    last_optimal_links = None


    # Step 2: Formulate Constraints
    for link in links:
        node_gateway_links[link.node].add((link.gateway, link.power))

    # Step 3: Implement Exact Cover Algorithm using Depth-First Search (DFS)
    
    # Flag to indicate whether an exact solution was found
    exact_solution_found = False

    # Step 3: Implement Exact Cover Algorithm using Depth-First Search (DFS)
    def dfs(node_set, selected_gateways, selected_nodes, selected_links, total_power):
        nonlocal min_power_consumption, optimal_gateways, optimal_nodes, optimal_links, exact_solution_found
        nonlocal last_optimal_gateways, last_optimal_nodes, last_optimal_links

        # Check if all nodes are covered
        if not node_set:
            exact_solution_found = True
            
            if total_power < min_power_consumption:
                min_power_consumption = total_power
                optimal_gateways = selected_gateways.copy()
                optimal_links = selected_links.copy()
                optimal_nodes = selected_nodes.copy()
            
            return
        else:
            if last_optimal_gateways is None or total_power < min_power_consumption:
                last_optimal_gateways = selected_gateways.copy()
                last_optimal_nodes = selected_nodes.copy()
                last_optimal_links = selected_links.copy()

        # Find the best node to cover
        best_node = max(node_set, key=lambda node: node.data / node_power_budget[node])
        
        if DEBUG:
            print("Current Node Set:", [node.name for node in node_set])
            print("Selected Gateways:", [gateway.name for gateway in selected_gateways])
            print("Selected Nodes:", [node.name for node in selected_nodes])
            print("Selected Links:", [(node.name, gateway.name) for node, gateway in selected_links])
            print("Total Power:", total_power)
            print("------------------------")

        # Try covering the best node with each available gateway
        for gateway, power in node_gateway_links[best_node]:
            if power <= node_power_budget[best_node] and gateway_capacity[gateway] >= best_node.data:
                node_set.remove(best_node)
                selected_nodes.add(best_node)
                selected_links.add((best_node, gateway))
                selected_gateways.add(gateway)
                node_power_budget[best_node] -= power
                gateway_capacity[gateway] -= best_node.data
                dfs(node_set, selected_gateways, selected_nodes, selected_links, total_power + gateway.power_setup)
                node_set.add(best_node)
                selected_nodes.remove(best_node)  # Remove the selected node when backtracking
                selected_links.remove((best_node, gateway))  # Remove the selected link when backtracking
                if gateway in selected_gateways:
                    selected_gateways.remove(gateway)
                node_power_budget[best_node] += power
                gateway_capacity[gateway] += best_node.data
        

    dfs(node_set, selected_gateways, selected_nodes, selected_links, 0)

    if optimal_gateways:
        return optimal_gateways, optimal_nodes, optimal_links
    else:
        return last_optimal_gateways, last_optimal_nodes, last_optimal_links


In [910]:
import random

# # Define parameters for generating random data
# num_nodes = 5  # Number of nodes
# num_gateways = 7  # Number of gateways
# max_data = 100  # Maximum data to transmit by a node
# max_power = 100  # Maximum power budget of a node
# max_capacity = 200  # Maximum storage capacity of a gateway
# max_power_setup = 50  # Maximum power setup of a gateway
# max_power_transmission = 30  # Maximum power transmission from node to gateway

# nodes = [Node(f'n{i}', random.randint(1, max_data), random.randint(1, max_power)) for i in range(1, num_nodes + 1)]

# # Generate random gateways
# gateways = [Gateway(f'g{i}', random.randint(1, max_capacity), random.randint(1, max_power_setup)) for i in range(1, num_gateways + 1)]

# # Generate random links
# links = []
# for node in nodes:
#     for gateway in gateways:
#         if random.random() < 0.5:  # Probability of having a link
#             links.append(Link(node, gateway, random.randint(1, max_power_transmission)))

num_nodes = 20
num_gateways = 5

# Generate nodes
nodes = [Node(f'n{i}', random.randint(1, 20), random.randint(1, 20)) for i in range(1, num_nodes + 1)]

# Generate gateways
gateways = [Gateway(f'g{i}', random.randint(1, 20), random.randint(20, 100)) for i in range(1, num_gateways + 1)]

# Generate links
links = []
for node in nodes:
    for gateway in gateways:
        # Ensure the link is feasible
        power_needed = random.randint(1, min(node.power, gateway.power_setup))
        links.append(Link(node, gateway, power_needed))





In [911]:
def main():
    selected_solution = exact_cover(nodes, gateways, links)
    if selected_solution is None:
        print("No solution exists.")
    else:
        selected_gateways, selected_nodes, selected_links = selected_solution
        total_power = sum(gateway.power_setup for gateway in selected_gateways)
        print("Selected Gateways:", [gateway.name for gateway in selected_gateways])
        print("Total Power Consumption:", total_power)
        print("Total Selected Gateways:", len(selected_gateways))
        print("Total Selected Nodes:", len(selected_nodes))
        print("Total Selected Links:", len(selected_links))

if __name__ == "__main__":
    main()


Selected Gateways: ['g2']
Total Power Consumption: 1
Total Selected Gateways: 1
Total Selected Nodes: 5
Total Selected Links: 5
